# Differentially Private Deep Learning:
## How to use the PyTorch Opacus Library

This script is my implementation of the following [tutorial](https://blog.openmined.org/differentially-private-deep-learning-using-opacus-in-20-lines-of-code/).


Relevant papers include: 

- https://arxiv.org/abs/2206.07758 - demonstrate that you can reverse engineer a neural network and see the initial images

 ### Step 1: Import libraries

In [1]:
import torch
from torchvision import datasets, transforms

In [2]:
import numpy as np
from opacus import PrivacyEngine
from tqdm import tqdm # progress bar

### Step 2: Load MNIST Data

In [8]:
train_loader = torch.utils.data.DataLoader(datasets.MNIST('../mnist',
			   train=True, download=True,
               transform=transforms.Compose([transforms.ToTensor(),
               transforms.Normalize((0.1307,), (0.3081,)),]),),
               batch_size=64, shuffle=True, num_workers=1,
               pin_memory=True)

In [4]:
test_loader = torch.utils.data.DataLoader(datasets.MNIST('../mnist',
			  train=False,
              transform=transforms.Compose([transforms.ToTensor(),
              transforms.Normalize((0.1307,), (0.3081,)),]),),
              batch_size=1024, shuffle=True, num_workers=1,
              pin_memory=True)

### Step 3: Creating a PyTorch Neural Network Classification Model and Optimizer

In [10]:
model = torch.nn.Sequential(torch.nn.Conv2d(1, 16, 8, 2, padding=3),
							torch.nn.ReLU(),
                            torch.nn.MaxPool2d(2, 1), 
                            torch.nn.Conv2d(16, 32, 4, 2), 
                            torch.nn.ReLU(), 
                            torch.nn.MaxPool2d(2, 1), 
                            torch.nn.Flatten(), 
                            torch.nn.Linear(32 * 4 * 4, 32), 
                            torch.nn.ReLU(), 
                            torch.nn.Linear(32, 10))

optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

### Step 4: Attaching a Differential Privacy Engine to the Optimizer

In [23]:
# this does not work - not sure why

privacy_engine = PrivacyEngine(
    model, 
    batch_size=64, 
    sample_size=60000, 
    alphas=range(2,32), 
    noise_multiplier=1.3, 
    max_grad_norm=1.0,)

privacy_engine.attach(optimizer)

TypeError: PrivacyEngine.__init__() got an unexpected keyword argument 'batch_size'

In [22]:
# modified version - this works but unclear that it is the correct way forward

privacy_engine = PrivacyEngine()

model, optimizer, train_loader = privacy_engine.make_private(
    module=model, 
    optimizer=optimizer, 
    data_loader=train_loader, 
    noise_multiplier=1.0, 
    max_grad_norm=1.0,)

/home/caroline/.pyenv/versions/3.11.3/lib/python3.11/site-packages/opacus/privacy_engine.py:142: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(


In [ ]:
def train(model, train_loader, optimizer, epoch, device, delta):
    model.train()
    criterion = torch.nn.CrossEntropyLoss()
    losses = []
    for _batch_idx, (data, target) in enumerate(tqdm(train_loader)):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    
    epsilon, best_alpha = 
    	optimizer.privacy_engine.get_privacy_spent(delta)
        
    print(
        f"Train Epoch: {epoch} \t"
        f"Loss: {np.mean(losses):.6f} "
        f"(ε = {epsilon:.2f}, δ = {delta}) for α = {best_alpha}")
    
for epoch in range(1, 11):
    train(model, train_loader, optimizer, epoch, device="cpu", delta=1e-5)